In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from scipy import stats,sparse
from sklearn.base import TransformerMixin
from datetime import datetime as dt
from math import isnan
from numpy import ma
import pickle as pickle
import xgboost as xgb
import time
from pandas import *
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier 

import json
from sklearn.metrics import roc_curve, auc
from re import sub
from collections import defaultdict
import scipy as sp

from sklearn.cross_validation import StratifiedKFold,cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import linear_model
from sklearn import metrics

#from ModelClassifier import PAClassifier,SGDSVMClassifier,XGBoostClassifier

In [2]:
with open('features/XGB_335Features Oct172015_044255_AUC_0p76115.p', 'rb') as fid1:
    xgb_goodfeat1 = pickle.load(fid1)
with open('features/XGB_482Features_backward_selection_XX1X2X3X4.p', 'rb') as fid2:
    xgb_goodfeat2 = pickle.load(fid2)
with open('features/XGB_5features_backward_selection_X5.p', 'rb') as fid3:
    xgb_goodfeat3 = pickle.load(fid3)
with open('features/SDGRF_Oct172015125634_AUC_0.755531555054_with_cat_variables.p', 'rb') as fid4:
    xgb_goodfeat4 = pickle.load(fid4)
with open('features/XGBoost_Oct172015212459_AUC_0.764813590574FeatNum199.p', 'rb') as fid5:
    xgb_goodfeat5 = pickle.load(fid5)

In [3]:
FeaturesSelected=list(set(list(xgb_goodfeat1) + xgb_goodfeat2 + xgb_goodfeat3 + list(xgb_goodfeat4) +list(xgb_goodfeat5)))

In [4]:
len(FeaturesSelected)

950

In [5]:
X=np.load('./pickledata/nxtrain_standard_original0.npy')
X1= np.load('./pickledata/nxtrain_standard_derived0.npy')
X2=pd.read_pickle("./pickledata/time_series_derived_standard_train2.dat")
X3=pd.read_pickle("./pickledata/time_series_original_standard_train2.dat")
X4=pd.read_pickle("./pickledata/cat_numeric_th60_standard_train2.dat")
X5=pd.read_pickle("./pickledata/cat_le_train2.dat") 
X=np.hstack((X,X1,X2,X3,X4,X5))

y=pickle.load(open("./pickledata/ytrain2.dat","rb"))
xtrain=X[:,FeaturesSelected]
del X,X1,X2,X3, X4, X5

X_test=np.load('./pickledata/nxtest_standard_original0.npy')
X_test1= np.load('./pickledata/nxtest_standard_derived0.npy')
X_test2=pd.read_pickle("./pickledata/time_series_derived_standard_test2.dat")
X_test3=pd.read_pickle("./pickledata/time_series_original_standard_test2.dat")
X_test4=pd.read_pickle("./pickledata/cat_numeric_th60_standard_test2.dat")
X_test5=pd.read_pickle("./pickledata/cat_le_test2.dat") 
X_test=np.hstack((X_test,X_test1,X_test2,X_test3,X_test4,X_test5))

xtest=X_test[:,FeaturesSelected]
del X_test,X_test1,X_test2,X_test3,X_test4,X_test5

In [6]:
num_boost_vals = 1000
eta_vals      = 0.01
max_depth_vals = 12
colsample_bytree_vals = 0.4
min_child_weight_vals = 3 #[3, 9] 3 is better
subsample_vals = 0.9
# gamma_vals = [0, 1]
lambda_vals = 0   # no L2
alpha_vals = 1 #l1

In [7]:
KFoldNumber=3

kfold = StratifiedKFold(y, KFoldNumber,shuffle=True, random_state=42)
ypreds_test_wl=pd.DataFrame()
ypreds_train_wl=pd.DataFrame()
ypreds_cvindex_wl=pd.DataFrame()
AUCValues=np.zeros(KFoldNumber)

kfoldnum=0

train_predicted=np.zeros(y.shape)

params = {'max_depth':max_depth_vals, 
          'eta':eta_vals, 
          'objective':'binary:logistic', 
          'subsample': subsample_vals,
          'colsample_bytree': colsample_bytree_vals,
          'eval_metric': 'auc',
          'min_child_weight': min_child_weight_vals, #3 for this computer
          'lambda': lambda_vals,
          'alpha':alpha_vals
         }

for train,cv in kfold:
    kfoldnum=kfoldnum+1
    cv_index=np.zeros(y.shape)

    dtrain = xgb.DMatrix(xtrain[train,:], label=y[train])
    clf = xgb.train(params=params,dtrain=dtrain, num_boost_round=num_boost_vals)    

    del dtrain

    dfulltrain = xgb.DMatrix(xtrain, label=y)
    preds_fulltrain = clf.predict(dfulltrain)   

    del dfulltrain

    dfulltest = xgb.DMatrix(xtest, label=np.zeros(xtest.shape[0]))
    preds_fulltest = clf.predict(dfulltest)

    del dfulltest

    preds_cv=preds_fulltrain[cv]
    cv_index[cv]=1

    AUC1=metrics.roc_auc_score(y[cv], preds_cv)

    ypreds_test_wl['CV'+ str(kfoldnum)] = preds_fulltest
    ypreds_train_wl['CV'+ str(kfoldnum)] = preds_fulltrain
    ypreds_cvindex_wl['CV'+ str(kfoldnum)] = cv_index
    AUCValues[kfoldnum-1]=float(AUC1)
#                 print(AUCValues)

trainPredict=(ypreds_train_wl*ypreds_cvindex_wl).sum(axis=1)
testPredictMean=ypreds_test_wl.mean(axis=1)
AUC1=metrics.roc_auc_score(y, trainPredict)
print(AUC1)

# TestPredict=pd.DataFrame()
# TestPredict['ID']=TestID
# TestPredict['Probability']=testPredictMean


str1=time.strftime("%b%d%Y%H%M%S", time.localtime())
filename='xgb_'+str1+'_AUC_0p'+ str(int(AUC1.mean()*1e6))+ '.p'
pickle.dump((trainPredict,testPredictMean,ypreds_train_wl,ypreds_test_wl,ypreds_cvindex_wl,AUCValues), 
            open("gridsearch_final/" + filename,'wb'), protocol =2) 

print('Model prediction done!')
    

0.787790420035
Model prediction done!
